# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/weather_data.csv")
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,thompson,55.74,-97.86,-2.20,84,40,3.36,CA,1607659730
1,lucera,41.51,15.34,44.60,81,75,4.70,IT,1607659926
2,bengkulu,-3.80,102.27,82.69,70,89,12.10,ID,1607659928
3,punta arenas,-53.15,-70.92,44.60,81,0,5.82,CL,1607659929
4,rikitea,-23.12,-134.97,77.22,79,60,14.23,PF,1607659930


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat','Lng']]
humidity = cities_df['Humidity']
print(len(humidity))

562


In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
vacation_df = cities_df.loc[(cities_df['Max Temp']>71) & (cities_df['Wind Speed']<10) \
                            & (cities_df['Max Temp']<79) & (cities_df['Humidity']<70) \
                            & (cities_df['Cloudiness']<10)]

In [18]:
### Hotel Map
# * Store into variable named `hotel_df`.
# * Add a "Hotel Name" column to the DataFrame.
# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.

In [19]:
vacation_df['Hotel'] = ""

#vacation_df.drop(vacation_df[vacation_df['City'] == "agadez"].index, inplace=True)
vacation_df

<ipython-input-19-7c40fe4ad9b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vacation_df['Hotel'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
155,new norfolk,-42.78,147.06,71.01,36,0,1.99,AU,1607659890,
170,wawa,9.90,4.42,72.88,40,0,2.82,NG,1607660121,
353,kedougou,12.56,-12.17,75.20,50,0,2.24,SN,1607660260,
358,mecca,21.43,39.83,71.04,40,0,4.88,SA,1607660329,
402,ladario,-19.00,-57.60,77.40,58,0,3.00,BR,1607660354,
444,launceston,-41.45,147.17,71.01,40,0,3.36,AU,1607660394,


In [20]:
#check iterrow logic
for index, row in vacation_df.iterrows():
    location_marker =  "\"%s\"" % f"{row['Lat']}, {row['Lng']}"
    print(location_marker)
    print(f"{index}: {row['City']}")
    #vacation_df.loc[index, 'location'] = str(location)
    #vacation_df['location'] = vacation_df['location'].astype(str)

"-42.78, 147.06"
155: new norfolk
"9.9, 4.42"
170: wawa
"12.56, -12.17"
353: kedougou
"21.43, 39.83"
358: mecca
"-19.0, -57.6"
402: ladario
"-41.45, 147.17"
444: launceston


In [21]:
#Find hotels
for index, row in vacation_df.iterrows():
    # set up a parameters dictionary
    location_marker = f"{row['Lat']}, {row['Lng']}"
    params = {
        "location": location_marker,
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    try:
        response = requests.get(base_url, params=params)
        places_data = response.json()
        #print(json.dumps(places_data, indent=4, sort_keys=True))
        print(places_data["results"][0]["name"])
        vacation_df.loc[index, 'Hotel'] = places_data["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        vacation_df.loc[index, 'Hotel'] = "No Hotel found"

The Shingles Riverside Cottages


C:\Users\melanie.nolker\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.
Relais de Kédougou
Hotel Fairmont Makkah Clock Royal Tower
Hotel Fazenda 4 cantos
Hotel Grand Chancellor Launceston


In [22]:
vacation_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
155,new norfolk,-42.78,147.06,71.01,36,0,1.99,AU,1607659890,The Shingles Riverside Cottages
170,wawa,9.90,4.42,72.88,40,0,2.82,NG,1607660121,No Hotel found
353,kedougou,12.56,-12.17,75.20,50,0,2.24,SN,1607660260,Relais de Kédougou
358,mecca,21.43,39.83,71.04,40,0,4.88,SA,1607660329,Hotel Fairmont Makkah Clock Royal Tower
402,ladario,-19.00,-57.60,77.40,58,0,3.00,BR,1607660354,Hotel Fazenda 4 cantos
444,launceston,-41.45,147.17,71.01,40,0,3.36,AU,1607660394,Hotel Grand Chancellor Launceston


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations_markers = vacation_df[["Lat", "Lng"]] 

In [24]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(locations_markers,info_box_content=hotel_info)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,
                                 point_radius = 4)

# Display figure
fig.add_layer(markers)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))